In [1]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [2]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

object_detection/protos/input_reader.proto: warning: Import object_detection/protos/image_resizer.proto but not used.


In [3]:
%%bash 
cd models/research
pip install .

Processing /home/workspace/models/research
  Running setup.py bdist_wheel for object-detection: started
  Running setup.py bdist_wheel for object-detection: finished with status 'done'
  Stored in directory: /tmp/pip-ephem-wheel-cache-gts_6_zs/wheels/9d/7a/58/d93aa7fa8ddef7adbbbb03eeb76e7e03c419b28165ba5c05c4
Successfully built object-detection


In [4]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

from timeit import default_timer as timer

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [5]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

/opt/conda/lib/python3.6/site-packages/object_detection/utils/visualization_utils.py:29: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/opt/conda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/conda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/opt/conda/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 497, in start
    self.io_loop.start()
  File "/opt/conda/lib/

In [6]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

In [7]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))

  return model

In [8]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [9]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  start = timer()
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)
  end = timer()
  elapsed=(end - start)
  #print(elapsed)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict,elapsed

In [10]:
def show_inference(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
    image_np = np.array(image_path)#Image.open(image_path))
  # Actual detection.
    output_dict,elapsed = run_inference_for_single_image(model, image_np)
    #print(output_dict['detection_classes'])
    #print(output_dict['detection_scores'])
    count_person=0
    conf_avg=0
    i=0
    for clase in output_dict['detection_classes']: # Output shape is 1x1x100x7
        if (clase==1):
            conf = output_dict['detection_scores'][i]
            if conf >= 0.6:
                count_person+=1
                conf_avg+=conf
        i+=1

  # Visualization of the results of a detection.
  #vis_util.visualize_boxes_and_labels_on_image_array(
  #    image_np,
  #    output_dict['detection_boxes'],
  #    output_dict['detection_classes'],
  #    output_dict['detection_scores'],
  #    category_index,
  #    instance_masks=output_dict.get('detection_masks_reframed', None),
  #    use_normalized_coordinates=True,
  #    line_thickness=8)

  #display(Image.fromarray(image_np))
    return elapsed,conf_avg,count_person

In [11]:
model_name = "mask_rcnn_resnet101_atrous_coco_2018_01_28"
masking_model = load_model(model_name)

661905408/661899639 [==============================] - 9s 0us/step
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
import cv2
counter=0
elapsed_prom=0
conf_avg=0
count_frame_person_total=0
conf_prom=0

cap = cv2.VideoCapture("/home/workspace/resources/prueba1.mp4" )
cap.open("/home/workspace/resources/prueba1.mp4" )  
while cap.isOpened():
    counter+=1
    flag, frame = cap.read()
    if not flag:
        break

    elapsed,conf_avg,count_person=show_inference(masking_model, frame)
    elapsed_prom=(elapsed_prom+elapsed)
    print(elapsed_prom)

    if(count_person>0):
        conf_prom+=conf_avg
        count_frame_person_total+=count_person
        
cap.release()
elapsed_prom=elapsed_prom/(counter-1)
conf_prom/=count_frame_person_total

83.76450613700035
205.91409600100042
319.79827159400065
433.2892979810008
498.12593154400065
553.2717948960008
607.8528822790008
662.9009645550009
717.7154962970008
772.4021072580008
827.2064683690005
882.3664767970004
937.3638513890005
992.4566201980006
1047.6879632870005
1102.3557608340002
1157.2068363590001
1211.42963921
1266.0380256930002
1320.6061073370001
1375.038468576
1429.53250115
1485.2623939270002
1563.75716058
1621.1166856869995
1696.3318142399994
1825.4906329319992
1940.7644334719998
2055.0944899590004


In [ ]:
print("Elapsed Average")
print(elapsed)
print("Accuracy Average")
print(conf_prom)